In [ ]:
import os, random, sys, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import tifffile as tff
import dask
import joblib

codeDirs = [r'V:/code/python/code', r'V:\Code\Python\code']
[sys.path.append(_) for _ in codeDirs]

import apCode.FileTools as ft
import apCode.volTools as volt
from joblib import Parallel, delayed
import apCode.behavior.FreeSwimBehavior as fsb
import apCode.behavior.headFixed as hf
import apCode.SignalProcessingTools as spt
# import apCode.geom as geom
import seaborn as sns
# import importlib
from apCode import util as util
import rsNeuronsProj.util as rsp
from apCode.machineLearning import ml as mlearn
from apCode.machineLearning.unet import model as my_model

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42


try:
    if __IPYTHON__:
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

# Setting seed for reproducability
seed = 143
random.seed = seed

print(time.ctime())

In [ ]:
#%% Path to excel sheet storing paths to data and other relevant info
dir_xls = r'Y:\Avinash\Projects\RS recruitment\Ablations'
file_xls = 'Ablation data summary.xlsx'


*In the cell below, I am ignoring fish data that was not tracked in MATLAB. There are about 6 such datasets, and these were not tracked because they are images collected under collimated light wherein the fish silhouette has nice contrast, but in terms of pixel intensity the fish's head is not distinguishable from the tail. I must visit these datasets at some point and modify my tracking scripts to deal with collimated light images. Alternatively, I could look into APT from the Branson lab*

In [ ]:
#%% Read xl file
xls = pd.read_excel(os.path.join(dir_xls, file_xls), sheet_name='RevisitedWithNN')
xls = xls.loc[xls.TrackedInMatlab == 1]
xls.head()

### *Filter to exclude collimated light datasets*

In [ ]:
xls = xls.loc[xls.Illumination!="collimated"]
xls.shape

In [ ]:
#%% Append xls with pxl size and image dimensions
paths_xls = np.array(xls.Path)
paths = rsp.remove_suffix_from_paths(paths_xls, suffix='proc')
paths_proc = rsp.add_suffix_to_paths(paths, suffix='proc')
# xls = xls.assign(path_proc = paths_proc)
%time img_props = rsp.get_img_props(paths, override_and_save=True)
pxlSizes, imgDims = zip(*[img_prop for img_prop in img_props])
pxlSizes, imgDims = np.array(pxlSizes), np.array(imgDims)

#%% Now assign values to xls
xls = xls.assign(pxlSize=pxlSizes, imgDims=list(imgDims), Path_proc=paths_proc)

In [ ]:
#%% Check for relationship between image dimension and pxl size
inds = np.where(imgDims[:,0] == 600)[0]
clrs = np.zeros_like(pxlSizes)
clrs[inds] = 1
plt.scatter(np.arange(len(pxlSizes)), pxlSizes, c=clrs, cmap='coolwarm')

### *Save xls with additional info*

In [ ]:
path_xls = os.path.join(dir_xls, file_xls.split('.')[0] + f'_{util.timestamp()}.pkl')
%time xls.to_pickle(path_xls)


### *Reload saved xls and continue...*

In [ ]:
file_xls = ft.findAndSortFilesInDir(dir_xls, ext='.pkl', search_str ='Ablation')[-1]
path_xls = os.path.join(dir_xls, file_xls)
xls = pd.read_pickle(path_xls)

### *Load pre-trained unet (in this case, loss func = focal_loss and optimizer = rmsprop)*

In [ ]:
dir_unet = r'Y:\Avinash\Ablations and Behavior'
file_unet = ft.findAndSortFilesInDir(dir_unet, ext='h5', search_str='trainedU')[-1]
print(f'Loading U-net: {file_unet}')
path_unet = os.path.join(dir_unet, file_unet)

unet = mlearn.loadPreTrainedUnet(path_unet)
print(f'U net dims = {unet.input_shape}')
print(time.ctime())


In [ ]:
motion_thresh_perc = None
trackedWithNN = np.zeros((xls.shape[0], ))
for iPath in np.arange(51, xls.shape[0]):
    path_ = xls.iloc[iPath].Path
    path_imgs = rsp.remove_suffix_from_paths([path_], suffix='proc')[0]
    path_proc = os.path.join(path_imgs, 'proc')
    fn = ft.findAndSortFilesInDir(path_proc, ext='h5', search_str='procData')
    if len(fn)>0:
        fn = fn[-1]
        try:
            with h5py.File(os.path.join(path_proc, fn), mode='r') as hFile:
                if 'tailAngles' not in hFile:
                    print(f'{iPath+1}/{len(xls)}')
                    print(path_imgs)
                    tic = time.time()
                    hFilePath = fsb.tail_angles_from_raw_imgs_using_unet(path_imgs, unet, 
                                                                         motion_threshold_perc=motion_thresh_perc)
                    print(int(time.time()-tic), 's')
                    trackedWithNN[iPath]=1
        except:
            fn = f'procData_{util.timestamp()}.h5'
            with h5py.File(os.path.join(path_proc, fn), mode='a') as hFile:
                print(f'{iPath+1}/{len(xls)}')
                print(path_imgs)
                tic = time.time()
                hFilePath = fsb.tail_angles_from_raw_imgs_using_unet(path_imgs, unet, 
                                                                     motion_thresh_perc=motion_thresh_perc)
                print(int(time.time()-tic), 's')
                trackedWithNN[iPath]=1
      
    else:
        print('Tracking from scratch')
        fn = f'procData_{util.timestamp()}.h5'
        with h5py.File(os.path.join(path_proc, fn), mode='a') as hFile:
            print(f'{iPath+1}/{len(xls)}')
            print(path_imgs)
            tic = time.time()
            hFilePath = fsb.tail_angles_from_raw_imgs_using_unet(path_imgs, unet, 
                                                                 motion_thresh_perc=motion_thresh_perc)
            print(int(time.time()-tic), 's')
            trackedWithNN[iPath]=1
        
        print(f'Skipped path # {iPath}\n {path_proc}')

### *Create a larger dataframe that also contains tail angles info*

In [ ]:
%%time
#%% Create another dataframe with tail angles info and merge with original dataframe using common FishIdx column

dataFrame = xls.copy() # Make a copy and work with that

paths_retrack = []
dict_list = dict(FishIdx = [], tailAngles = [], tailAngles_tot = [])
for iPath in range(len(dataFrame)):
    df_ = dataFrame.iloc[iPath]
    path_ = df_.Path_proc
    fn = ft.findAndSortFilesInDir(path_, search_str='procData', ext='h5')
    if len(fn)>0:
        fn = fn[-1]
        path_hFile = os.path.join(path_, fn)
        with h5py.File(path_hFile, mode='r') as hFile:
            try:
                keys = hFile.keys()
                key = 'tailAngles'
                if key in keys:
                    ta = np.array(hFile[key]).transpose()
                    dict_list['FishIdx'].append(df_.FishIdx)
                    dict_list['tailAngles'].append(ta)
                    dict_list['tailAngles_tot'].append(ta[-1])
                else:
                    print(f'No tailAngles in path # {iPath}\n {path_hFile}')
                    paths_retrack.append(path_)
            except Exception:
                print(f'Cannot read path # {iPath},  hdf file\n {path_hFile}')
                paths_retrack.append(path_)
    else:
        print(f'No hdf file found for path # {iPath}\n {path_}')
        paths_retrack.append(path_)
df_now = pd.DataFrame(dict_list)
dataFrame_orig = pd.merge(dataFrame, df_now, on='FishIdx')

In [ ]:
#%% Extract, filter, and clean tail angles
ta_fish = np.array(dataFrame_orig.tailAngles)
ta = np.concatenate(ta_fish, axis=1)

# This will allow me to put the cleaned tail angles back into the dataframe
fishIdx = dataFrame_orig.FishIdx
fid_list = []
for fid, taf in zip(fishIdx, ta_fish):
    fid_ = fid*np.ones((taf.shape[1], ))
    fid_list.append(fid_)
fishIdx_ser = np.concatenate(fid_list, axis=0)

ta_flt = [dask.delayed(spt.chebFilt)(_, 1/500, (5, 60), btype='bandpass') for _ in ta]
%time ta_flt = np.array(dask.compute(*ta_flt))

%time ta_clean, _, svd = hf.cleanTailAngles(ta_flt)
ta_tot = ta_clean[-1]
# ta_tot = spt.chebFilt(ta_tot, 1/500, (5, 60), btype='bandpass')

In [ ]:
#%% Load the trained model
dir_group = r'Y:\Avinash\Projects\RS recruitment\GCaMP imaging\Group'
file_model = 'gmm_svd-3_env_pca-9_gmm-20_20200129-18.pkl' 
gmm_model = joblib.load(os.path.join(dir_group, file_model))

%time labels, features = gmm_model.predict(ta_clean)
%time envelopes = spt.emd.envelopesAndImf(ta_tot)['env']
env_max, env_diff = envelopes['max'], envelopes['diff']

In [ ]:
#%% Plot to check out what the labels look like
xl = (10e3, 15e3)

inds = np.arange(*xl,dtype='int')
plt.figure(figsize=(20, 10))
# plt.plot(ta_clean[-1])
x = np.arange(len(inds))
# plt.plot(x, ta_tot[inds])
plt.plot(x, env_diff[inds])
clrs = plt.cm.tab20(labels[inds])
for x_ in x[::10]:
    plt.scatter(x_, env_diff[inds][x_], c=clrs[x_][None, :], marker=r'${}$'.format(labels[inds][x_]), s=200)
plt.xlim(0, len(inds))
# plt.xlim(2000, 3e3)
plt.ylim(-100, 100)

### *Labels-based detection of swim onsets and offsets followed by modificaton of dataframe to include episodes of tail angles*

In [ ]:
### %%time
lbls_start = [11, 18]
lbls_end = [19, 2]
tKer = 200e-3
Fs = 500
dur_swim = 800e-3
dur_preSwim = 100e-3
thr_swimOn = 0.5 # Z-score units for convolved label-based swim start vector


# Detect swim onsets based on labels
vec_start = np.zeros_like(labels)
vec_end = np.zeros_like(labels)
for lbl in lbls_start:
    vec_start[np.where(labels==lbl)]=1
for lbl in lbls_end:
    vec_end[np.where(labels==lbl)]=1
nKer = (tKer)*Fs 
vec_start = spt.causalConvWithSemiGauss1d(vec_start, nKer)
vec_start /= vec_start.std()
vec_end = spt.causalConvWithSemiGauss1d(vec_end, nKer)
vec_end /= vec_end.std()

onsets_swim = spt.levelCrossings(vec_start, thr = thr_swimOn)[0]
onsets_swim = onsets_swim-int(dur_preSwim*Fs)
offsets_swim = onsets_swim + int(dur_swim*Fs)

# Remove episodes too close to beginning
inds_del = np.where(onsets_swim < 0)
onsets_swim = np.delete(onsets_swim, inds_del, axis=0)
offsets_swim = np.delete(offsets_swim, inds_del, axis=0)

# Remove episodes too close to end
inds_del = np.where(offsets_swim >= len(vec_start))
onsets_swim = np.delete(onsets_swim, inds_del, axis=0)
offsets_swim = np.delete(offsets_swim, inds_del, axis=0)


# Episodic tail angles to be incorporated into the data
ta_tot_ep, ta_ep, env_diff_ep, env_max_ep = [], [], [], []
fishIdx_ep = []
for on, off in zip(onsets_swim, offsets_swim):
    ta_tot_ep.append(ta_tot[on:off])
    ta_ep.append(ta_clean[:,on:off])
    env_diff_ep.append(env_diff[on:off])
    env_max_ep.append(env_max[on:off])
    fishIdx_ep.append(int(fishIdx_ser[on]))
dict_list = dict(tailAngles = ta_ep, tailAngles_tot = ta_tot_ep, tailAngles_env_max=env_max_ep,
                 tailAngles_env_diff=env_diff_ep, FishIdx=fishIdx_ep, episodeNum=list(range(len(onsets_swim))))
df_now = pd.DataFrame(dict_list)

dataFrame_drop = dataFrame_orig.drop(columns=['tailAngles', 'tailAngles_tot'])
dataFrame = pd.merge(dataFrame_drop, df_now, on='FishIdx')
print(dataFrame.columns)

In [ ]:
dataFrame.head()

### *Pull envelopes for each of the ablation groupd and compare control vs ablated*

In [ ]:
print(f'Dataframe cols = {dataFrame.columns}')
print(f'\nAblation Groups = {np.unique(dataFrame.AblationGroup)}')


In [ ]:
figDir = r'Y:\Avinash\Projects\RS recruitment\Figures'
saveDir = os.path.join(figDir, f'{util.timestamp("day")}')
if not os.path.exists(saveDir):
    os.mkdir(saveDir)

In [ ]:
ablationGrp = 'mHom'

df_now = dataFrame.loc[dataFrame.AblationGroup==ablationGrp]
df_ctrl = df_now.loc[df_now.Treatment=='ctrl']
df_abl = df_now.loc[df_now.Treatment=='abl']

env_ctrl = np.array([np.array(_) for _ in df_ctrl.tailAngles_env_max])*4
env_abl = np.array([np.array(_) for _ in df_abl.tailAngles_env_max])*(4)*np.sqrt(env_abl.shape[0])/np.sqrt(env_ctrl.shape[0])

mu_ctrl = env_ctrl.mean(axis=0)
mu_abl = env_abl.mean(axis=0)

env_ctrl -= env_ctrl[:, 0][:, None]
env_abl -= env_abl[:, 0][:, None]

# env_ctrl -= np.median(mu_ctrl[:20])
# env_abl -= np.median(env_abl[:20])

mu_ctrl = env_ctrl.mean(axis=0)
mu_abl = env_abl.mean(axis=0)

sem_ctrl = env_ctrl.std(axis=0)/np.sqrt(env_ctrl.shape[0])
sem_abl = env_abl.std(axis=0)/np.sqrt(env_abl.shape[0])

# Plot
plt.figure(figsize=(20, 10))

t = 1000*np.arange(len(mu_ctrl))*(1/Fs)
plt.fill_between(t, mu_ctrl-sem_ctrl, mu_ctrl+sem_ctrl, color=plt.cm.tab10(0), alpha=0.5, label='Control')
plt.fill_between(t, mu_abl-sem_abl, mu_abl+sem_abl, color=plt.cm.tab10(1), alpha=0.5, label='Ablated')
plt.axvline(x=75, ls='--', c='k', alpha=0.5)
plt.xlim(t[0]+10, t[-1])
plt.ylim(-40, 140)
plt.yticks([0, 50])
plt.xticks([100, 200])
plt.legend(fontsize=20)
plt.title(ablationGrp, fontsize=24)

plt.savefig(os.path.join(saveDir, f'Fig-{util.timestamp()}_Envelopes ctrl vs abl_{ablationGrp}' + '.pdf'),
            dpi='figure')

plt.savefig(os.path.join(saveDir, f'Fig-{util.timestamp()}_Envelopes ctrl vs abl_{ablationGrp}' + '.png'),
            dpi='figure')

In [ ]:
ablationGrp = 'intermediateRS'

df_now = dataFrame.loc[dataFrame.AblationGroup==ablationGrp]
df_ctrl = df_now.loc[df_now.Treatment=='ctrl']
df_abl = df_now.loc[df_now.Treatment=='abl']

env_ctrl = np.array([np.array(_) for _ in df_ctrl.tailAngles_env_max])*4
env_abl = np.array([np.array(_) for _ in df_abl.tailAngles_env_max])*4


mu_ctrl = env_ctrl.mean(axis=0)
mu_abl = env_abl.mean(axis=0)

# env_ctrl -= env_ctrl[:, 0][:, None]
# env_abl -= env_abl[:, 0][:, None]

env_ctrl -= mu_ctrl[:12].mean()
env_abl -= env_abl[:12].mean()

mu_ctrl = env_ctrl.mean(axis=0)
mu_abl = env_abl.mean(axis=0)

sem_ctrl = env_ctrl.std(axis=0)/np.sqrt(env_ctrl.shape[0])
sem_abl = env_abl.std(axis=0)/np.sqrt(env_abl.shape[0])

# Plot
plt.figure(figsize=(20, 10))

t = 1000*np.arange(len(mu_ctrl))*(1/Fs)
plt.fill_between(t, mu_ctrl-sem_ctrl, mu_ctrl+sem_ctrl, color=plt.cm.tab10(0), alpha=0.5, label='Control')
plt.fill_between(t, mu_abl-sem_abl, mu_abl+sem_abl, color=plt.cm.tab10(1), alpha=0.5, label='Ablated')

plt.xlim(t[0]+10, t[-1])
plt.ylim(-20, 140)
plt.axvline(x=75, ls='--', c='k', alpha=0.5)
plt.yticks([0, 50])
plt.xticks([100, 200])
plt.legend(fontsize=20)
plt.title(ablationGrp, fontsize=24)

plt.savefig(os.path.join(saveDir, f'Fig-{util.timestamp()}_Envelopes ctrl vs abl_{ablationGrp}' + '.pdf'),
            dpi='figure')

plt.savefig(os.path.join(saveDir, f'Fig-{util.timestamp()}_Envelopes ctrl vs abl_{ablationGrp}' + '.png'),
            dpi='figure')

In [ ]:
ablationGrp = 'ventralRS'

df_now = dataFrame.loc[dataFrame.AblationGroup==ablationGrp]
df_ctrl = df_now.loc[df_now.Treatment=='ctrl']
df_abl = df_now.loc[df_now.Treatment=='abl']

env_ctrl = np.array([np.array(_) for _ in df_abl.tailAngles_env_diff])*4.4
env_abl = np.array([np.array(_) for _ in df_ctrl.tailAngles_env_diff])*5.75

mu_ctrl = env_ctrl.mean(axis=0)
mu_abl = env_abl.mean(axis=0)

env_ctrl -= env_ctrl[:, 0][:, None]
env_abl -= env_abl[:, 0][:, None]

# env_ctrl -= np.median(mu_ctrl[:5])
# env_abl -= np.median(env_abl[:5])

mu_ctrl = env_ctrl.mean(axis=0)
mu_abl = env_abl.mean(axis=0)

sem_ctrl = env_ctrl.std(axis=0)/np.sqrt(env_ctrl.shape[0])
sem_abl = env_abl.std(axis=0)/np.sqrt(env_abl.shape[0])

# Plot
plt.figure(figsize=(20, 10))

t = 1000*np.arange(len(mu_ctrl))*(1/Fs)
plt.fill_between(t, mu_ctrl-sem_ctrl, mu_ctrl+sem_ctrl, color=plt.cm.tab10(0), alpha=0.5, label='Control')
plt.fill_between(t, mu_abl-sem_abl, mu_abl+sem_abl, color=plt.cm.tab10(1), alpha=0.5, label='Ablated')
plt.axvline(x=75, ls='--', c='k', alpha=0.5)
plt.xlim(t[0]+10, t[-1])
plt.ylim(-40, 140)
plt.yticks([0, 50])
plt.xticks([100, 200])
plt.legend(fontsize=20)
plt.title(ablationGrp, fontsize=24);

plt.savefig(os.path.join(saveDir, f'Fig-{util.timestamp()}_Envelopes ctrl vs abl_{ablationGrp}' + '.pdf'),
            dpi='figure')

plt.savefig(os.path.join(saveDir, f'Fig-{util.timestamp()}_Envelopes ctrl vs abl_{ablationGrp}' + '.png'),
            dpi='figure')

In [ ]:
#%% Create a dataframe that includes swimming information (fish position, swim distance, etc)
%time df= rsp.append_fishPos_to_xls(xls)

#%% Compute onset latencies
%time df = rsp.detect_noisy_trials(df)

df_den = df.loc[df.noisyTrlInds_swimVel==0]
df = rsp.append_latency_to_df(df_den, zThr=0.5)
inds_del = np.where(np.isnan(df.onsetLatency))[0]
df = df.set_index(np.arange(df.shape[0]))
df = df.drop(index=inds_del)
df = df.assign(onsetLatency_log=np.log(df.onsetLatency))


## Adjust swim distances and velocities to make consistent across ablation groups

In [ ]:
sf_dist = 1.25
sf_vel = 1.4

inds = np.where(df.AblationGroup != 'mHom')[0]
swimDist_total = np.array(df.swimDist_total)
swimDist_total[inds] = swimDist_total[inds]*sf_dist

swimVel_max = np.array(df.swimVel_max)
swimVel_max[inds] = swimVel_max[inds]*sf_vel


df = df.assign(swimDist_total_adj=swimDist_total,\
               swimDist_total_adj_log=np.log2(swimDist_total),
               swimVel_max_adj=swimVel_max,\
               swimVel_max_adj_log=np.log2(swimVel_max))


# Categorical plots

In [ ]:
#%% Total swim distance
saveDir = f'Y:\Avinash\Projects\RS recruitment\Figures\{util.timestamp("day")}'
#%% Filter extreme values
q_max = 99
q_min = 5

if not os.path.exists(saveDir):
    os.mkdir(saveDir)
thr_low = np.percentile(df.swimDist_total, q_min)
thr_high = np.percentile(df.swimDist_total, q_max)

df_sub = df.loc[(df.swimDist_total > thr_low) & (df.swimDist_total < thr_high)]


figName = f'Fig_{util.timestamp()}_Total swim distance_ctrl-vs-abl_mHom_interRS_ventralRS'
fh = sns.catplot(data= df_sub, x = 'AblationGroup', y = 'swimDist_total_adj',
                 kind = 'boxen', sharey=True, hue = 'Treatment', dodge = True,
                 order = ['mHom', 'intermediateRS', 'ventralRS'],
                 aspect=2)
fh.savefig(os.path.join(saveDir, figName + '_boxen.pdf'), format = 'pdf', dpi = 'figure')
fh.savefig(os.path.join(saveDir, figName + '_boxen.png'), format = 'png', dpi = 'figure')

figName = f'Fig_{util.timestamp()}_Total swim distance_log_ctrl-vs-abl_mHom_interRS_ventralRS'
fh = sns.catplot(data= df_sub, x = 'AblationGroup', y = 'swimDist_total_adj_log',
                 kind = 'boxen', sharey=True, hue = 'Treatment', dodge = True,
                 order = ['mHom', 'intermediateRS', 'ventralRS'],
                 aspect=2)
fh.savefig(os.path.join(saveDir, figName + '_boxen.pdf'), format = 'pdf', dpi = 'figure')
fh.savefig(os.path.join(saveDir, figName + '_boxen.png'), format = 'png', dpi = 'figure')



plt.show()
thr_low = np.percentile(df.swimVel_max, q_min)
thr_high = np.percentile(df.swimVel_max, q_max)

df_sub = df.loc[(df.swimVel_max > thr_low) & (df.swimVel_max < thr_high)]

figName = f'Fig_{util.timestamp()}_Max swim vel_ctrl-vs-abl_mHom-interRS-ventralRS_boxen'
fh = sns.catplot(data= df_sub, x = 'AblationGroup', y = 'swimVel_max_adj',
                 kind = 'boxen', sharey=True, hue = 'Treatment',\
                 dodge = True, order = ['mHom', 'intermediateRS', 'ventralRS'],
                 aspect=2)
fh.savefig(os.path.join(saveDir, figName + '_boxen.pdf'), format = 'pdf', dpi = 'figure')
fh.savefig(os.path.join(saveDir, figName + '_boxen.png'), format = 'png', dpi = 'figure')


figName = f'Fig_{util.timestamp()}_Max swim vel_log_ctrl-vs-abl_mHom-interRS-ventralRS'
fh = sns.catplot(data= df_sub, x = 'AblationGroup', y = 'swimVel_max_adj_log',
                 kind = 'boxen', sharey=True, hue = 'Treatment',\
                 dodge = True, order = ['mHom', 'intermediateRS', 'ventralRS'],
                 aspect=2)
fh.savefig(os.path.join(saveDir, figName + '_boxen.pdf'), format = 'pdf', dpi = 'figure')
fh.savefig(os.path.join(saveDir, figName + '_boxen.png'), format = 'png', dpi = 'figure')



In [ ]:

#%% Plot latencies
minLat = 5
maxLat = 30

    
fn = f'Fig-{util.timestamp()}_Onset latencies_log_ctrl-vs-abl_mHom_interRS_ventralRS'
df_now = df.loc[(df.onsetLatency >= minLat) & (df.onsetLatency <= maxLat)]

fh = sns.catplot(data= df_now, x='AblationGroup', y='onsetLatency_log', hue='Treatment',
                 order=['mHom', 'intermediateRS', 'ventralRS'],
                 margin_titles=True, kind='boxen', dodge=True, aspect=2)
fh.savefig(os.path.join(saveDir, fn + '_boxen.pdf'))
fh.savefig(os.path.join(saveDir, fn + '_boxen.png'))

plt.show()

fh = sns.catplot(data= df_now, x='AblationGroup', y='onsetLatency_log', hue='Treatment',
                 order=['mHom', 'intermediateRS', 'ventralRS'],
                 margin_titles=True, kind='swarm', dodge=True, aspect=2, alpha=0.5)
fh.savefig(os.path.join(saveDir, fn + '_swarm.pdf'))
fh.savefig(os.path.join(saveDir, fn + '_swarm.png'))

plt.show()
fh = sns.catplot(data= df_now, x='AblationGroup', y='onsetLatency_log', hue='Treatment',
                 order=['mHom', 'intermediateRS', 'ventralRS'],
                 margin_titles=True, kind='box', dodge=True, aspect=2)
fh.savefig(os.path.join(saveDir, fn + '_box.pdf'))
fh.savefig(os.path.join(saveDir, fn + '_box.png'))

# Save to CSV/pickle file file

In [ ]:
%%time
saveDir = r'Y:\Avinash\Projects\RS recruitment\Ablations\CSV files'
fn = f'Dataframe with swim trajectories and global swim parameters_{util.timestamp()}'
%time df.to_csv(os.path.join(saveDir,fn + '.csv'), columns=df.keys(), index = False)
%time df.to_pickle(os.path.join(saveDir, fn + '.pkl'))

## Resume analysis

In [ ]:
dir_csv = r'Y:\Avinash\Projects\RS recruitment\Ablations\CSV files'
fn = ft.findAndSortFilesInDir(dir_csv, search_str='Dataframe with swim trajectories', ext = '.pkl')[-1]
df = pd.read_pickle(os.path.join(dir_csv, fn))
print(df.columns)

## Resume after loading the saved csv file

In [ ]:
saveDir = r'Y:\Avinash\Projects\RS recruitment\Ablations\CSV files'
fn = ft.findAndSortFilesInDir(saveDir, ext = 'csv', search_str= 'mHomologs_intermediateRS_ventralRS')
if len(fn)>0:
    fn = fn[-1]
df = pd.read_csv(os.path.join(saveDir,fn))
df

In [ ]:
#%% Bend amplitude vs bend number

figDir = f'Y:\Avinash\Projects\RS recruitment\Figures\{util.timestamp()}'
if not os.path.exists(figDir):
    os.mkdir(figDir)
nBends = 10

# plt.style.use(('seaborn-paper', 'seaborn-whitegrid'))
amps = np.array(df.bendAmp_rel)
amps = np.delete(amps, np.where(np.isnan(amps)))
amp_low = np.percentile(amps,5)
amp_high = np.percentile(amps,99)

pers = np.array(df.bendInt)
pers = np.delete(pers, np.where(np.isnan(pers)))
per_low = np.percentile(pers,5)
per_high = np.percentile(pers,95)

df_now = df.loc[(df.bendAmp_rel>= amp_low) & (df.bendAmp_rel <= amp_high) & (df.bendInt >= per_low) & \
               (df.bendInt <= per_high) & (df.bendNum>=0) & (df.bendNum <=nBends)]

# df_now = df.loc[(df.bendAmp_rel>= 40) & (df.bendAmp_rel <= amp_high) & \
#                 (df.bendInt <= 60) & (df.bendNum>=0) & (df.bendNum <=10)]

plt.style.use(('seaborn-ticks', 'seaborn-paper'))
fh = sns.catplot(data = df_now, x = 'bendNum', y = 'bendAmp_rel', hue = 'treatment', col = 'ablationGroup',\
            col_order=['mHom', 'intermediateRS', 'ventralRS'], aspect = 1.5, dodge = True,\
            sharex=True, sharey=True, kind = 'point', palette=plt.cm.tab10(np.arange(10)),\
           hue_order = ['ctrl','abl'], height = 3, ci  =99, scale = 0.5)

figName = f'Fig_{util.timestamp()}_Bend amplitude vs bend number for ctrl and abl_mHom_intermediate_ventral'
fh.savefig(os.path.join(figDir, figName + '.pdf'), format = 'pdf', dpi = 'figure')
fh.savefig(os.path.join(figDir, figName + '.png'), format = 'png', dpi = 'figure')

print(f'Saved at {figDir}')

In [ ]:
#%% Bend interval
nBends = 10

df_now = df.loc[(df.bendAmp_rel>= amp_low) & (df.bendAmp_rel <= amp_high) & \
                (df.bendInt <= per_high) & (df.bendNum>=0) & (df.bendNum <=nBends)]


# plt.style.use(('seaborn-white', 'seaborn-paper'))
fh = sns.catplot(data = df_now, x = 'bendNum', y = 'bendInt', hue = 'treatment', col = 'ablationGroup',\
            col_order=['mHom', 'intermediateRS', 'ventralRS'], aspect = 1.5, dodge = True,\
            sharex=True, sharey=True, kind = 'point', palette=plt.cm.tab10(np.arange(10)),\
           hue_order = ['ctrl','abl'], height = 3, ci  =99, scale = 0.5)

figName = f'Fig_{util.timestamp()}_Bend interval vs bend number for ctrl and abl_mHom_intermediate_ventral'
fh.savefig(os.path.join(figDir, figName + '.pdf'), format = 'pdf', dpi = 'figure')
fh.savefig(os.path.join(figDir, figName + '.png'), format = 'png', dpi = 'figure')

print(f'Saved at {figDir}')

In [ ]:
df.columns